In [2]:
import requests

D:\Anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [9]:
import pandas as pd

BASE_URL = "https://api.nb.no/ngram/db2"
BASE_URL1 = "https://api.nb.no/ngram/db1"

In [10]:
BASE_URL = "https://api.nb.no/dhlab"
BASE_URL1 = "https://api.nb.no/dhlab"

In [11]:
def get_metadata(urns = None):
    params = locals()
    r = requests.post(f"{BASE_URL}/get_metadata", json = params)
    return r.json()

In [12]:
urns = list(document_corpus(doctype = 'digibok', from_year = 1920, to_year = 1930, limit = 300).urn)

NameError: name 'document_corpus' is not defined

In [55]:
print(len(urns))

300


In [60]:
pd.DataFrame(get_metadata(urns = urns[:10]))

,urn,title,authors,oaiid,sesamid,isbn10,city,timestamp,year,publisher,langs,subjects,ddc,genres,literaryform,doctype
0,URN:NBN:no-nb_digibok_2014101506007,Flukten til Island,"Engelstad , Arne",oai:nb.bibsys.no:999523623094702202,a98620ec3a5304fa1bf51841bc9a7876,,Oslo,19250101,1925,Grøndahl,nob,,,novel,Skjønnlitteratur,digibok
1,URN:NBN:no-nb_digibok_2016012548121,Udvalgte romancer og sange,"Kjerulf , Halfdan",oai:nb.bibsys.no:991507527254702202,21b975750ac682c0d735af8cbe6a5d55,,Leipzig,19280101,1928,Brandstetter,nob,sanger / romanser,783.242,,Uklassifisert,digibok
2,URN:NBN:no-nb_digibok_2010110806068,Odysseen,"Homerus / Østbye , P. (Peter)",oai:nb.bibsys.no:990309816704702202,25d8aad9c5028a65b7edf1ec2a2a2f80,,Kristiania,19220101,1922,Gyldendal,nob,,,,Uklassifisert,digibok
3,URN:NBN:no-nb_digibok_2008040300060,Salmonsens Konversationsleksikon. B. 9 : Friel...,"Blangstrup , Chr.",oai:nb.bibsys.no:999203609714702202,a75aa349bc931759f0b29eb53440b7f8,,København,19200101,1920,Schultz,dan,Leksika / generelle / skrifter / konversasjons...,038.81,,Uklassifisert,digibok
4,URN:NBN:no-nb_digibok_2006082400067,Gjennem Kaukasus til Volga,"Nansen , Fridtjof",oai:nb.bibsys.no:999603051654702202,c1447c5a04d3aaa1cf365f63e248da07,,,19290101,1929,Dybwad,nob,Reiseskildringer / Reiseskildringer,,,Uklassifisert,digibok
5,URN:NBN:no-nb_digibok_2013070408051,Ditlev Ahlefeldts Erindringer : 1617-1660,"Ahlefeldt , Ditlev",oai:nb.bibsys.no:999423089644702202,c61b48963e6415a994acc1bc58f2b5bb,,København,19220101,1922,Koppel,dan,,,,Uklassifisert,digibok
6,URN:NBN:no-nb_digibok_2014121708109,Lov om forsikringsavtaler av 6. juni 1930 : me...,"Bugge , Nicolay L.",oai:nb.bibsys.no:998121481854702202,79b8733aaf4e5f623120b79485ebd336,,Oslo,19300101,1930,Aschehoug,nob,forsikringsavtalelover / kommentarer / norge,,,Uklassifisert,digibok
7,URN:NBN:no-nb_digibok_2007121910007,Gesandten : drama i 4 akter,"Aall , Herman Harris",oai:nb.bibsys.no:990504319354702202,15ca33af147a3a63f65d570ba8b10b18,,Stockholm,19200101,1920,Chelius,swe,,,,Uklassifisert,digibok
8,URN:NBN:no-nb_digibok_2017112048044,Die Kristallstruktur der wasserfreien Alaune R...,"Vegard , L. / Maurstad , Alf",oai:nb.bibsys.no:999720241074702202,4d56978f1217f2db418a137777177c0a,,Oslo,19280101,1928,I kommisjon hos Dybwad,ger,,,,Uklassifisert,digibok
9,URN:NBN:no-nb_digibok_2010062106078,Vor gudstjeneste efter den nye alterbok,"Jensen , S. Bretteville (Sigurd)",oai:nb.bibsys.no:999722125664702202,db96b426a7040a6b47aa02d58da15318,,,19200101,1920,Luthersk bokmission,nob,Liturgi / Teologi / alterbøker / norske / kirk...,,,Uklassifisert,digibok


In [13]:
def konk(urns=None, word=None, before=5, after=5, only_urn=True):
    params = locals()
    #print(params)
    r = requests.post("https://api.nb.no/ngram/urnkonk", json = params)
    return r.json()

def meta_natbib(children = False, yearfrom = "", yearto = "", lang = None, author = None, title = None, subtitle = None, publisher = None, subject = None, topic = None, marctuples = None, limit=20):
    params = locals()
    #print(params)
    r = requests.post("https://api.nb.no/ngram/get_urns", json = params)
    return r.json()

def metadata(urns=None, marctuples = [(100,1,' ', 'a'),(260, ' ',' ', 'c'), (245,1,3,'a'),(245,1,0,'a'),(245,1,3,'b'),(245,1,0,'b'), (650,7,' ','a'), (653,' ',' ','a')]):
    params = locals()
    #print(params)
    r = requests.post("https://api.nb.no/ngram/metadata", json=params)
    return r.json()

In [14]:
def document_corpus(doctype = None, author = None, freetext = None, from_year = None, to_year = None, from_timestamp = None, to_timestamp = None, title = None, ddk = None, subject = None, lang = None, limit = None):
    """ Fetch a corpus based on metadata - doctypes are digibok, digavis, digitidsskrift"""
    
    parms = locals()
    params = {x:parms[x] for x in parms if not parms[x] is None }
    if "ddk" in params:
        params["ddk"]  = "^" + params['ddk'].replace('.', '"."')
        
    r=requests.post(BASE_URL + "/build_corpus", json=params)
    
    return pd.DataFrame(r.json())

In [15]:
c = document_corpus(freetext="hamsun")

In [25]:
def frequencies(urns = None, cutoff = 0, words = None):
    params = locals()
    print(params)
    r = requests.post(BASE_URL1 + "/frequencies", json = params)
    result = r.json()
    structure = {u[0][0] : dict([tuple(x[1:]) for x in u]) for u in result if u != []}
    return structure

In [17]:
def find_urns(docids = None):
    params = locals()
    r = requests.post(BASE_URL1 + "/find_urn", json = params)
    return r.json()

In [31]:
def get_reference(corpus = 'digavis', from_year = 1950, to_year = 1955, lang = 'nob', limit = 100000):
    params = locals()
    r = requests.get(BASE_URL + "/reference_corpus", params = params)
    if r.status_code == 200:
        result = r.json()
    else:
        result = []
    return pd.DataFrame(result, columns = ['word', 'freq']).set_index('word')

In [26]:
frequencies(list(c.urn))

{'urns': ['URN:NBN:no-nb_digibok_2007082001024', 'URN:NBN:no-nb_digibok_2015012708298', 'URN:NBN:no-nb_digibok_2013071908027', 'URN:NBN:no-nb_digibok_2015102208046', 'URN:NBN:no-nb_digibok_2018042509509', 'URN:NBN:no-nb_digibok_2008051504035', 'URN:NBN:no-nb_digibok_2015072108261', 'URN:NBN:no-nb_digibok_2009011504011', 'URN:NBN:no-nb_digibok_2012091808258', 'URN:NBN:no-nb_digibok_2008121704075'], 'cutoff': 0, 'words': None}


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [19]:
get_reference(limit = 100000)

NameError: name 'get_reference' is not defined

In [20]:
import dhlab_v2 as d2

In [21]:
urns = list(d2.document_corpus(from_year = 2000, to_year = 2002, limit = 10).urn)

In [31]:
result = frequencies(urns = urns, cutoff = 100, words = "spise sover skrive .".split())

{'urns': ['URN:NBN:no-nb_digavis_ostlandsposten_null_null_20010317_120_65_1', 'URN:NBN:no-nb_digavis_fremover_null_null_20020205_100_31_1', 'URN:NBN:no-nb_digibok_2017030348111', 'URN:NBN:no-nb_digavis_verdensgang_null_null_20020121_1_20_1', 'URN:NBN:no-nb_digitidsskrift_2018101981128_001', 'URN:NBN:no-nb_digitidsskrift_2020012781428_001', 'URN:NBN:no-nb_digibok_2009020404097', 'URN:NBN:no-nb_digibok_2015112508222', 'URN:NBN:no-nb_digavis_jaerbladet_null_null_20000616_52_67_1', 'URN:NBN:no-nb_digavis_gjengangeren_null_null_20011227_151_299_1'], 'cutoff': 100, 'words': ['spise', 'sover', 'skrive', '.']}


In [32]:
result

{'URN:NBN:no-nb_digitidsskrift_2020012781428_001': {'.': 1280,
  'skrive': 4,
  'sover': 1},
 'URN:NBN:no-nb_digavis_fremover_null_null_20020205_100_31_1': {'.': 2681,
  'spise': 1},
 'URN:NBN:no-nb_digavis_verdensgang_null_null_20020121_1_20_1': {'.': 2994,
  'skrive': 7,
  'sover': 3},
 'URN:NBN:no-nb_digibok_2009020404097': {'.': 1118},
 'URN:NBN:no-nb_digibok_2015112508222': {'.': 2373, 'skrive': 21, 'spise': 2},
 'URN:NBN:no-nb_digibok_2017030348111': {'.': 2},
 'URN:NBN:no-nb_digavis_ostlandsposten_null_null_20010317_120_65_1': {'.': 5015,
  'skrive': 1,
  'sover': 2,
  'spise': 1},
 'URN:NBN:no-nb_digavis_gjengangeren_null_null_20011227_151_299_1': {'.': 1617,
  'sover': 1},
 'URN:NBN:no-nb_digitidsskrift_2018101981128_001': {'.': 2653, 'skrive': 6},
 'URN:NBN:no-nb_digavis_jaerbladet_null_null_20000616_52_67_1': {'.': 4336,
  'skrive': 8,
  'sover': 6,
  'spise': 1}}

In [15]:
df = pd.DataFrame(result)

In [24]:
fu = find_urns(docids = list(df.columns))

In [28]:
pd.DataFrame.from_dict(fu, orient = 'index', columns = ['urn'])

,urn
200135263,URN:NBN:no-nb_digavis_arbeidetsrett_null_null_...
200382347,URN:NBN:no-nb_digavis_dagsavisen_null_null_200...
200553644,URN:NBN:no-nb_digavis_fiskeribladet_null_null_...
200769595,URN:NBN:no-nb_digavis_harstadtidende_null_null...
201457838,URN:NBN:no-nb_digavis_romerikesblad_null_null_...
201568584,URN:NBN:no-nb_digavis_sognavis_null_null_20010...
201752010,URN:NBN:no-nb_digavis_tromsfolkeblad_null_null...
201824775,URN:NBN:no-nb_digavis_ullensakerblad_null_null...
202574785,URN:NBN:no-nb_digavis_nordhordland_null_null_2...
300065604,URN:NBN:no-nb_digitidsskrift_2021011981065_001


In [79]:
import dhlab.nbtext as nb

In [93]:
t = nb.book_corpus(author='ibsen%', title='%dukke%', period=(1800, 2010))

In [95]:
list(t.urn)

[2011010609040,
 2010031510001,
 2010112608074,
 2016031009565,
 2009061000024,
 2010082422025,
 2011072908024,
 2010083123001,
 2015120808042,
 2012091409046,
 2012082309582,
 2007082204013,
 2013010808064,
 2007091301037,
 2008112000060,
 2011040608127,
 2012080709541,
 2011082409020,
 2011030820014,
 2010083022002]

In [111]:
metadata(urns = list([str(x) for x in t.urn]), marctuples = [(100,1,' ', 'a'), (245,1,3,'a')])

{'2007082204013': {'100-1- -a': 'Ibsen, Henrik',
  '245-1-3-a': 'Et dukkehjem :',
  'lang': 'nob',
  'year': 1991},
 '2007091301037': {'100-1- -a': 'Ibsen, Henrik',
  '245-1-3-a': 'Et dukkehjem',
  'lang': 'nob',
  'year': 1989},
 '2008112000060': {'100-1- -a': 'Ibsen, Henrik',
  '245-1-3-a': 'Et dukkehjem',
  'lang': 'nob',
  'year': 2001},
 '2009061000024': [],
 '2010031510001': {'100-1- -a': 'Ibsen, Henrik',
  '245-1-3-a': 'Et dukkehjem :',
  'lang': 'nob',
  'year': 1879},
 '2010082422025': [],
 '2010083022002': {'100-1- -a': 'Ibsen, Henrik',
  '245-1-3-a': 'Et dukkehjem og fire andre skuespil',
  'lang': 'nob',
  'year': 2006},
 '2010083123001': {'100-1- -a': 'Ibsen, Henrik',
  '245-1-3-a': 'Et dukkehjem ; Gengangere ; Vildanden',
  'lang': 'nob',
  'year': 1989},
 '2010112608074': {'100-1- -a': 'Ibsen, Henrik',
  '245-1-3-a': 'Et dukkehjem',
  'lang': 'nob',
  'year': 1995},
 '2011010609040': {'100-1- -a': 'Ibsen, Henrik',
  '245-1-3-a': 'Et dukkehjem :',
  'lang': 'nob',
  'year